# Import Packages and Read Data

This assumes there is a subdirectory in your current working directory called `data` that contains the Excel files.

In [1]:
import pandas as pd
from collections import defaultdict


deferred_factors = pd.read_excel('data/deferred_factors.xlsx',
                                 usecols=['Age', 'Combined_Factor'],
                                 index_col='Age',
                                 )
salary_increase_rate = pd.read_excel('data/salary_increase_rate.xlsx')
inforce_data = pd.read_excel('data/inforce_2022_p31.xlsx')

deferred_factors.columns = [col_name.lower() for col_name in deferred_factors.columns]
salary_increase_rate.columns = [col_name.lower() for col_name in salary_increase_rate.columns]
inforce_data.columns = [col_name.lower() for col_name in inforce_data.columns]


## Define Global Variables

### Increase Rates
`INCREASE_RATES` is an array of salary increase rates, where the index of the array is the years of service minus 1.

### Deferred Factors
`DEFERRED_FACTORS` is a dictionary of combined factors, mapping `age` -> `combined_factor`. For any age not contained in the dictionary, `combined_factor` is defaulted to 1.

### Accrued Benefit Factor
An assumption. Set to `0.0182`.

### PVAB Factor
An assumption. Set to `12.5`

In [2]:
INCREASE_RATES = salary_increase_rate['increase'].values
DEFERRED_FACTORS = defaultdict(lambda: 1, deferred_factors['combined_factor'].to_dict())
ACCRUED_BENEFIT_FACTOR = 0.0182
PVAB_FACTOR = 12.5

## Calculate FAS

The below code block defines a function call `calculate_fas` which calculates the Five Year Average Salary of a state pension contributor. This will be used to calculate their Accrued Benefit.

In [3]:
from src.python.calculate_fas import calculate_fas


### Example FAS Calculations

The function `calculate_fas` contains a parameter `years_ahead` which allows you to calculate a contributor's `FAS` years in advance. 

In the below example, we first calculate the `FAS` for a contributer with 19 years of experience (`yos`) and a current compensation of $100,000 (`comp`). The resulting value is stored in the `fas_example` variable.

Next, we calculate the contributor's prior year compensation (`prior_year_comp`) by leveraging the `INCREASE_RATES` array. We then calculate the `FAS` for this contributor at `yos = 18` and `comp = prior_year_comp`; however, we set `years_ahead = 1`. This should result in the same number obtained in `fas_example`.

In [4]:
yos = 19
comp = 100000
fas_example = round(calculate_fas(yos, comp, INCREASE_RATES), 2)
prior_year_comp = comp / (1 + INCREASE_RATES[yos - 1])
fas_example_next_year = round(calculate_fas(yos - 1, prior_year_comp, INCREASE_RATES, years_ahead=1), 2)
assert fas_example == fas_example_next_year, f"{'${:,.2f}'.format(fas_example)} != {'${:,.2f}'.format(fas_example_next_year)}"
print(f"{'${:,.2f}'.format(fas_example)} == {'${:,.2f}'.format(fas_example_next_year)}")

$93,387.90 == $93,387.90


### Add FAS calculations DataFrame

Now we can add `fas` and `fas_next_year` columns to our dataframe.

In [5]:
inforce_data['fas'] = inforce_data.apply(lambda row: calculate_fas(yos=int(row['yos']), 
                                                                   comp=row['comp'],
                                                                   increase_rates=DEFERRED_FACTORS), axis=1)
inforce_data['fas_next_year'] = inforce_data.apply(lambda row: calculate_fas(yos=int(row['yos']), 
                                                                             comp=row['comp'],
                                                                             increase_rates=DEFERRED_FACTORS, 
                                                                             years_ahead=1), 
                                                                             axis=1)
inforce_data.head()

,age,count,comp,yos,fas,fas_next_year
0,65,368,79921,35,65993.350237,73131.001880
1,57,310,75555,35,62388.203065,69135.932321
2,52,751,74812,32,63886.843506,69515.371341
3,52,4437,74659,27,66615.590422,70788.114805
4,62,530,73281,35,60510.487841,67055.128799


## Accrued Benefit & PVAB calculations

We can now leverage the `fas` calculations to calculate the Accrued Benefit and PVAB.

In [6]:
inforce_data['accrued_benefit'] = inforce_data['yos'] * inforce_data['fas'] * ACCRUED_BENEFIT_FACTOR
inforce_data['pvab'] = inforce_data['accrued_benefit'] * inforce_data['age'].map(DEFERRED_FACTORS) * PVAB_FACTOR
inforce_data['accrued_benefit_next_year'] = (inforce_data['yos'] + 1) * inforce_data['fas'] * \
    ACCRUED_BENEFIT_FACTOR
inforce_data['pvab_next_year'] = inforce_data['accrued_benefit_next_year'] * \
    (inforce_data['age'] + 1).map(DEFERRED_FACTORS) * PVAB_FACTOR
inforce_data['pvab_increase'] = inforce_data['pvab_next_year'] - inforce_data['pvab']
inforce_data.head()

,age,count,comp,yos,fas,fas_next_year,accrued_benefit,pvab,accrued_benefit_next_year,pvab_next_year,pvab_increase
0,65,368,79921,35,65993.350237,73131.001880,42037.764101,525472.051261,43238.843075,540485.538440,15013.487179
1,57,310,75555,35,62388.203065,69135.932321,39741.285352,278684.336508,40876.750648,307788.354226,29104.017718
2,52,751,74812,32,63886.843506,69515.371341,37207.697658,183455.282476,38370.438210,202916.408474,19461.125997
3,52,4437,74659,27,66615.590422,70788.114805,32734.901133,161401.831134,33947.304879,179525.319616,18123.488483
4,62,530,73281,35,60510.487841,67055.128799,38545.180754,386895.634024,39646.471633,427978.915610,41083.281587
